In [5]:
import numpy as np
import pandas as pd
import librosa
import sklearn
import os

In [6]:
# names = []
# Letter_names = ["M", "S", "B", "Y"]
# for letter in Letter_names:
#     for index in range(15):
#         names.append({'file':"{0}_open_({1}).wav".format(letter, index), 'label':"{0}".format(letter)})
# file_name = pd.DataFrame(names)
# print(file_name)

In [7]:
Names = os.listdir("AllRecords//Others")
Others_Names_df = pd.DataFrame(Names, columns={'Paths'})
Others_Names_df.insert(1, "Label", len(Names) *[0])
print(Others_Names_df)

               Paths  Label
0     a_open_(1).wav      0
1    a_open_(10).wav      0
2    a_open_(11).wav      0
3    a_open_(12).wav      0
4    a_open_(13).wav      0
..               ...    ...
152   r_open_(5).wav      0
153   r_open_(6).wav      0
154   r_open_(7).wav      0
155   r_open_(8).wav      0
156   r_open_(9).wav      0

[157 rows x 2 columns]


In [11]:
#Getting Team information
team_members = ['Sherif']
iterator = 0
Team_df = pd.DataFrame()
labels = pd.DataFrame()
for member in team_members:
    iterator += 1
    Team = os.listdir("AllRecords/Team/{0}/Open_the_door".format(member))
    Team_df = Team_df.append(Team,ignore_index=True)
    #Team_df.insert(1, "Label", [iterator]*len(Team))
    labels = labels.append([iterator]*len(Team),ignore_index=True)
# Team_df.insert(1, "Label", labels)
# Data = {
#     "Paths": names,
#     "Labels": labels
# }
# Team_df = pd.DataFrame(Data)
labels = labels.rename(columns={0:"Labels"})
Team_df = Team_df.rename(columns={0:"Paths"})
# print(labels)
# print(Team_df)
Team_df = pd.concat([Team_df,labels], axis=1)
print(Team_df)

             Paths  Labels
0  s_open_ (1).wav       1
1  s_open_ (2).wav       1
2  s_open_ (3).wav       1
3  s_open_ (4).wav       1
4  s_open_ (5).wav       1


C:\Users\mahmo\AppData\Local\Temp\ipykernel_6316\546527831.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Team_df = Team_df.append(Team,ignore_index=True)
C:\Users\mahmo\AppData\Local\Temp\ipykernel_6316\546527831.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  labels = labels.append([iterator]*len(Team),ignore_index=True)


In [34]:
def Extract_features(paths, whichone, dowhat = ''):
    # path = os.listdir("AllRecords/{0}/{1}/{2}".format(whichone, dowhat, paths["Paths"]))
    try:
        path = os.path.join(os.path.abspath('AllRecords/{0}/{1}').format(whichone, dowhat)+ ('\\') +str(paths['Paths']))
        Voices_frequencies, sample_rate = librosa.load(path, res_type = "kaiser_fast")
    except:
        print(paths["Paths"])
    print(path)
    mfccs = np.maen(librosa.feature.mfcc(Voices_frequencies, sample_rate, n_mfcc = 40).T, axis = 0)
    mel = np.mean(librosa.feature.melspectrogram(Voices_frequencies, sample_rate).T, axis = 0)
    tonnetz = np.mean(librosa.feature.tonnetz(librosa.effects.harmonic(Voices_frequencies), sample_rate).T, axis = 0)
    frequencies_in_stft = abs(librosa.stft(Voices_frequencies))
    chroma = np.mean(librosa.feature.chroma_stft(frequencies_in_stft, sample_rate).T, axis = 0)
    contrast = np.mean(librosa.feature.spectral_contrast(frequencies_in_stft, sample_rate).T, axis = 0)
    label = paths['Labels']
    return mfccs, mel, tonnetz, chroma, contrast, label
# def extract_features(files):
    
#     # Sets the name to be the path to where the file is in my computer
#     file_name = os.path.join(os.path.abspath('all')+'/'+str(files.file))

#     # Loads the audio file as a floating point time series and assigns the default sample rate
#     # Sample rate is set to 22050 by default
#     X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 

#     # Generate Mel-frequency cepstral coefficients (MFCCs) from a time series 
#     mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)

#     # Generates a Short-time Fourier transform (STFT) to use in the chroma_stft
#     stft = np.abs(librosa.stft(X))

#     # Computes a chromagram from a waveform or power spectrogram.
#     chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)

#     # Computes a mel-scaled spectrogram.
#     mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)

#     # Computes spectral contrast
#     contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)

#     # Computes the tonal centroid features (tonnetz)
#     tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
#     sr=sample_rate).T,axis=0)
        
    
#     # We add also the classes of each file as a label at the end
#     label = files.label

#     return mfccs, chroma, mel, contrast, tonnetz, label


In [38]:
mfccs = []
mel = []
tonnetz = []
chroma = []
contrast = []
label_enter = []
DataSet = pd.DataFrame()
files = Team_df['Paths']
print(Team_df['Paths'])
DataSet = files.apply(Extract_features, whichone="Team", dowhat = "Open_the_door")
# for i in range(len(Team_df)):
#     vmfcc, vmel, vtonnetz, vchroma, vcontrast, vlabel = Extract_features(Team_df[i], "Team", "Open_the_door")
#     mel.append(vmel)
#     tonnetz.append(vtonnetz)
#     chroma.append(vchroma)
#     contrast.append(vcontrast)
#     label_enter.append(vlabel)
#     mfccs.append(vmfcc)
# Data = {
#     "mfccs": mfccs,
#     "mel": mel,
#     "chroma": chroma,
#     "tonnetz": tonnetz,
#     "contrast": contrast,
#     "Labels": label_enter
# }
# DataSet = pd.DataFrame(Data)
# print(DataSet)

0    s_open_ (1).wav
1    s_open_ (2).wav
2    s_open_ (3).wav
3    s_open_ (4).wav
4    s_open_ (5).wav
Name: Paths, dtype: object


KeyError: 'Paths'